In [1]:
%run stdPackages.ipynb

The ```lpCompiler``` consists of two main databases and works as follows:
* Add an argument: This is a (key,type,value) tuple with key = name of constraint/argument, type = type of constraint/argument, and value the input to be parsed. Ideally, we would like three ways to add new arguments:
    1. Add a single, new argument.
    2. Adjust existing argument.
    3. Add an entire constraint from equation text (requires developing a compiler).
    
    Adding a new argument, we process the information and store in ```self.parameters``` (one container for each type in the ```lp``` program ```c, l, u, beq, bub, Aeq, Aub```). 
* Given the parameter structure, make functions that infer the global domains. 
* Given the parameter structure and global domains, create container for ```broadcasted``` parameter containers. These are sparse data structure that are filled with ```nan```, zeros, or whatever else default value is assumed. 
* Given broadcasted parameters, make ```__call__``` function that stack all parameters/constraints...

### 1. Add an argument to the model.
The arguments depend on what type of constraint we are considering.

* Adding coefficients on variables (```c,l,u```): Should be added as pd.Series with appropriate values, indices, and name of series (corresponding to variable name). To distinguish between adding ```c,l,u``` type of arguments, we specify a separate add function for each of them (e.g. ```addC```). More specifically:
    * ```name```: The name input is required as it used to add/adjust/subtract coefficient blocks after initial compilation.
    * ```value = None```: 
        * If the input is a scalar: The scalar is added with variable name ```varName```.
        * If the input is a pd.Series: The name of the series identifies the variable name. If ```None``` we default to the ```name```.
        * If the input is a list/tuple: The various components are summed/max/min if the block type is ```c,l,u``` respectively.
        * If the input is a dict: Add multiple coefficients (iterate through ```self.addC(k, **v)``` for ```k,v``` in ```input```). 
    * ```varName = None```: If the input is a scalar, this specifies the name of this scalar.

* Adding coefficients on constraint vectors (```beq, bub```): Added in a similar way as ```c,l,u```, with the difference that names relate to the relevant constraint.

* Adding coefficients on constraint matrices (```Aeq, Aub```): Add a constraint name and a collection of inputs. More specifically:
    * ```name```: The name of the constraint is required.
    * ```value```: dictionary with ```k``` = name of relevant variable and ```v``` depends on type of input
        * If ```v``` is a pd.Series: Coefficient is 
        * If ```v``` is a list/tuple: Add the various components by adding them. 

* Add a constraint (of type ```eq``` or ```ub```): Add ```b, A``` like coefficients:

Two compile methods: Compile with stacking or by summing/taking the max/taking the minimum depending on the type of parameter in question. The compile using stacking is much faster, but requires that relevant entries are not repeated. That means that a variable $x_s$ enters at most once for each $s$ in both $c,l,u$. 

```self.readDict```: NOTE Add this to the ```lpModel``` class instead.
* ```name```:  Name of symbol.
* ```value```: pd.Series or scalar.
* ```c```: Condition to be applied on ```value``` when adding coefficient. (through ```pyDatabases.adj.rc_pd```)
* ```alias```: Alias to be applied on ```value``` when adding coefficient. (through ```pyDatabases.adj.rc_pd```)
* ```lag```: Lag to be applied on ```value``` when adding coefficient. (through ```pyDatabases.adj.rc_pd```)
* ```fill_value=0```.

## 2. Sparse methods:

### 2.1. Init

In [2]:
size = 1000000

*Sparse series of zeros and otherwise default values:*

In [3]:
s = sparseSeries(np.zeros(size), index = None, name = None, fill_value = 0, dtype = None)

*Empty series (with fill_value as sparse data input):*

In [4]:
s = sparseEmptySeries(size, fill_value = 0, index = None, name = None)

#### *Sparse unstacking with specified index levels:*

*Create symbol with relevant index levels:*

In [5]:
indexVar = pd.MultiIndex.from_product([['varName'], range(size)], names = ['_vsymbol', '_vindex'])
indexEqs = pd.MultiIndex.from_product([['eqName'],  range(10, size+10)], names = ['_eqsymbol','_eqindex'])
index = pd.MultiIndex.from_frame(pd.concat([indexVar.to_frame(index=False), indexEqs.to_frame(index=False)],axis=1))
s = sparseSeries(np.random.rand(size), index = index)

Note that it takes longer to initialize the sparse series, but nothing that should be too prohibitively slow:

In [6]:
%%timeit
s = pd.Series(np.random.rand(size), index = index)

10.1 ms ± 685 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
s = sparseSeries(np.random.rand(size), index = index)

22.9 ms ± 676 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


*Map ```[_vsymbol, _vindex]``` indices to unique (row) identifiers:*

In [8]:
rowIds = s.groupby(['_vsymbol','_vindex']).ngroup().values
colIds = s.groupby(['_eqsymbol','_eqindex']).ngroup().values

*Create dataframe with sparse data input: This takes quite a lot of time:*

In [9]:
x = pd.DataFrame.sparse.from_spmatrix(sparse.coo_matrix((s.values, (rowIds, colIds)), shape = (len(rowIds), len(colIds))))

*Instead, we should only do the unstacking to a scipy matrix and parse this to the lp model:*

In [10]:
%%time
x = sparse.coo_matrix((s.values, (rowIds, colIds)), shape = (len(rowIds), len(colIds)))

Wall time: 507 ms


### 2.2. Sparse broadcast, sum, max, min, stack

*Create symbols for testing purposes:*

In [11]:
# Create some indices:
indices = {'x': pd.Index(range(1,11),  name = 'x'),
           'y': pd.Index(range(11,21), name = 'y'),
           'z': pd.Index([1,2], name = 'z')}
indices['xy'] = pd.MultiIndex.from_arrays([indices['x'], indices['y']])
# Add corresponding values - here just some ranges:
symbols = {'x': sparseSeries(range(1,11), indices['x'], dtype = np.int32),
           'y': sparseSeries(range(11,21), indices['y'], dtype=np.int32),
           'xy':sparseSeries(range(21,31), indices['xy'], dtype=np.int32)}

#### Broadcast

In [12]:
sparseBroadcast(1, symbols['x']) # 

NameError: name 'sparseBroadcast' is not defined

*Broadcast a series to another with non-overlapping domains. This returns the cartesian product index as default:*

In [ ]:
sparseBroadcast(symbols['x'], symbols['y'])

*Broadcast a series to another with overlapping domain. You can add ```fill_value``` as a keyword argument for the cases with missing values:*

In [ ]:
sparseBroadcast(symbols['x'], symbols['xy'], fill_value=0)

#### Sum

In [ ]:
ite = list(symbols.values())

Take the sum over an iterator with symbols that and broadcast to relevant domain if they do not match:

In [ ]:
sumIte(ite)